In [1]:
import os
import geopandas as gpd
import pandas as pd
import fiona
from earthai.geo import reproject_on_the_fly
import statistics
import matplotlib.pyplot as plt
from datetime import date
import re

In [2]:
# Enable fiona driver
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'

In [3]:
# read in annotations
df_annotations = pd.DataFrame()

for path, subdirs, files in os.walk('Completed'):
    for name in files:
        if name.endswith(".kml"): 
            full_path = os.path.join(path, name)
            name = full_path.split("/")[-2]
            plant = full_path.split("/")[-1]
            
            # Read file
            tmp = gpd.read_file(full_path, driver='KML')               
            tmp['Processed By'] = name.strip()
            tmp['tile_id'] = plant.split(".")[0].strip()

            df_annotations = df_annotations.append(tmp, ignore_index=True)
            
for path, subdirs, files in os.walk('Astraea to Review'):
    for name in files:
        if name.endswith(".kml"): 
            full_path = os.path.join(path, name)
            name = full_path.split("/")[-2]
            plant = full_path.split("/")[-1]
            
            # Read file
            tmp = gpd.read_file(full_path, driver='KML')               
            tmp['Processed By'] = name.strip()
            tmp['tile_id'] = plant.split(".")[0].strip()

            df_annotations = df_annotations.append(tmp, ignore_index=True)

In [4]:
# read in statuses
df_status = pd.DataFrame()

sets=['Annotator 1', 'Annotator 2']
for s in sets:
    tmp = pd.read_excel("Cement Plant Localization Worksheet - Group 1.xlsx", sheet_name=s)
    tmp = tmp[['tile_id','Plant_found','Class', 'Level','imagery_date','Processed By','Status']]
    
    df_status = df_status.append(tmp, ignore_index=True)
    
# convert to string
df_status['Processed By'] = df_status['Processed By'].astype(str)
df_status['tile_id'] = df_status['tile_id'].astype(str)

# to lower case
df_status['Processed By'] = df_status['Processed By'].str.lower()

# remove any leading/trailing spaces
df_status['tile_id'] = df_status['tile_id'].str.strip()
df_status['Processed By'] = df_status['Processed By'].str.strip()

# fix status
df_status.Status = df_status.Status.fillna('')
df_status.Status = df_status.Status.str.lower()
df_status.Status = df_status.Status.str.strip()

# fix status
df_status.Plant_found = df_status.Plant_found.fillna('')
df_status.Plant_found = df_status.Plant_found.str.lower()
df_status.Plant_found = df_status.Plant_found.str.strip()

In [5]:
# # check if merge worked
df = pd.merge(df_status, df_annotations, how='outer', left_on=['tile_id', 'Processed By'], right_on=['tile_id', 'Processed By'], indicator=True)

# df._merge.value_counts()
# df[df._merge == 'both']['Processed By'].value_counts()
# df[df._merge == 'right_only']['Processed By'].value_counts()
# df[df._merge == 'left_only']['Processed By'].value_counts()

In [6]:
df[df._merge == 'both']['Processed By'].value_counts()

isaack odhiambo otieno       155
benice wairimu ruga          118
maina lawrence irungu        118
marvin mwangi                112
rasoa simiyu                  63
fiona atieno                  47
emily nyawira waithera        38
damaris kwamboka okenyuri     35
clement omunga                27
vincent kipngetich            26
njoki muriithi                22
moses njau                    21
caroline kioko                15
erick karanja                 14
ephantus maina                 6
john oduor otieno              6
vincent omondi                 3
joshua gichuki mwangi          1
Name: Processed By, dtype: int64

In [7]:
df[df._merge == 'right_only']['Processed By'].value_counts()

Series([], Name: Processed By, dtype: int64)

In [8]:
df[df._merge == 'left_only']['Processed By'].value_counts()

nan                          1001
rasoa simiyu                  196
maina lawrence irungu         174
isaack odhiambo otieno        126
benice wairimu ruga           112
njoki muriithi                 91
damaris kwamboka okenyuri      89
marvin mwangi                  82
emily nyawira waithera         77
clement omunga                 57
vincent kipngetich             57
fiona atieno                   48
moses njau                     43
caroline kioko                 38
erick karanja                  28
ephantus maina                 10
john oduor otieno               5
joshua gichuki mwangi           1
Name: Processed By, dtype: int64

In [9]:
df_status_count = df_status.Status.value_counts().reset_index()
df_status_count.columns = ['status', 'count']
df_status_count['percent'] = (df_status_count['count'] / df_status_count['count'].sum())*100
df_status_count

,status,count,percent
0,no issues,2883,96.067977
1,plant already found,69,2.299234
2,no imagery,37,1.232922
3,unclear imagery,8,0.266578
4,cloudy imagery,4,0.133289


In [10]:
# join annotations and statuses
df = pd.merge(df_status, df_annotations, how='left', left_on=['tile_id', 'Processed By'], right_on=['tile_id', 'Processed By'])

# drop duplicates in case annotator accidentally exported the same annotation twice
df = df.drop_duplicates()

In [11]:
df['Acquisition_Date'] = pd.to_datetime(df.Description, errors='coerce')
df['Acquisition_Year'] = df['Acquisition_Date'].dt.year

df['Acquisition_Date2'] = pd.to_datetime(df.imagery_date, errors='coerce')
df['Acquisition_Year2'] = df['Acquisition_Date2'].dt.year

In [12]:
df_year_count = df.Acquisition_Year.value_counts().reset_index()
df_year_count.columns = ['year', 'count']
df_year_count['percent'] = (df_year_count['count'] / df_year_count['count'].sum())*100
df_year_count['year'] = df_year_count['year'].astype(int)
df_year_count = df_year_count[(df_year_count.year != 2026) & (df_year_count.year != 2107)]
df_year_count.sort_values('year')

,year,count,percent
11,2009,2,0.242131
10,2010,3,0.363196
9,2011,3,0.363196
6,2013,12,1.452785
8,2014,8,0.968523
7,2015,10,1.210654
5,2016,19,2.300242
4,2017,25,3.026634
2,2018,65,7.869249
1,2019,195,23.607748


### Quality Assurance

In [13]:
def check_annotation_quality(group):
    # add geometry type (point, linestring, polygon)
    group['geom_type'] = group.geometry.apply(lambda x: x.type)
    
    s = ""
    
    # check if point is missing
    if len(group[group.geom_type == 'Point']) == 0:
        s += "Point is missing; "
            
    # check acquisition dates
    for idx, row in group.iterrows():
        if pd.notnull(row.Acquisition_Date):
            if row.Acquisition_Date.year < 1900 or row.Acquisition_Date.year > 2021:
                s += "Date format is incorrect on one of annotations; "
        else:
            s += "Date is missing for annotated point; "
    
        if row.Name.strip() == '':
            s += "Name is missing for annotated point; "
            
    return s

In [14]:
# Remove previous feedback
!rm -r Completed/*
!rm -r CloudFactory\ to\ Review/*
!rm Completed.tar.gz
!rm CloudFactory\ to\ Review.tar.gz

In [15]:
# check if we have all the annotations
grouped = df.groupby(['tile_id'])
results_df = pd.DataFrame(columns=['tile_id', 'annotator1', 'issues1', 'annotator2', 'issues2'])

for name, group in grouped:
    
    res_dict = {}
    res_dict['tile_id'] = name    
    
    grouped2 = group.groupby(['tile_id', 'Processed By', 'Status', 'Plant_found', 'Class', 'Level', 'Acquisition_Date2', 'Acquisition_Year2'])
    idx = 1
    for name2, group2 in grouped2:
        s = ""
        if name2[2].strip() == '':
            s = "Status is missing; "
        elif name2[3].strip() == '':
            s = "Plant_found is missing; "
        elif name2[2] == 'no issues' and name2[3] == 'yes':
            if len(group2[group2.Name.notnull()]) == 0:
                s = "Annotations are missing; "
            else:
                s = check_annotation_quality(group2)
        
        if name2[4].strip() == '':
            s += "Class is missing; "
        if name2[5].strip() == '':
            s += "Level is missing; "
        if pd.isnull(name2[6]):
            s += "imagery_date is missing or format is incorrect; "
        else:
            if name2[7] < 1900 or name2[7] > 2021:
                s += "imagery_date format is incorrect; "

            
        res_dict['annotator{}'.format(idx)] = name2[1] 
        res_dict['issues{}'.format(idx)] = s 
        idx += 1
        
        tmp = df_annotations[(df_annotations["Processed By"] == name2[1]) & (df_annotations.tile_id == name)]
        
        if name2[1] != 'nan':
            if not os.path.isdir('Completed/{}'.format(name2[1])):
                os.mkdir('Completed/{}'.format(name2[1])) 
                os.mkdir('CloudFactory to Review/{}'.format(name2[1])) 

            if len(tmp) > 0:
                if s == '':
                    with fiona.drivers():
                        tmp.to_file('Completed/{}/{}.kml'.format(name2[1], name), driver='KML')
                else:
                    with fiona.drivers():
                        tmp.to_file('CloudFactory to Review/{}/{}.kml'.format(name2[1], name), driver='KML')
                        
    results_df = results_df.append(res_dict, ignore_index=True)

In [16]:
def check_overall(row):
    num_issues = 0
    if row.issues1 != '':
        num_issues += 1
    if row.issues2 != '':
        num_issues += 1
    return 2 - num_issues

results_df['num_completed'] = results_df.apply(check_overall,axis=1)

In [17]:
!pip install openpyxl

  Using cached openpyxl-3.0.7-py2.py3-none-any.whl (243 kB)
Processing /home/jovyan/.cache/pip/wheels/e2/bd/55/048b4fd505716c4c298f42ee02dffd9496bb6d212b266c7f31/et_xmlfile-1.0.1-py3-none-any.whl


In [18]:
import openpyxl as pxl

excel_book = pxl.load_workbook("Cement Plant Localization Worksheet - Group 1.xlsx")
with pd.ExcelWriter("Cement Plant Localization Worksheet - Group 1.xlsx", engine='openpyxl') as writer:
    writer.book = excel_book
    writer.sheets = {
        worksheet.title: worksheet
        for worksheet in excel_book.worksheets
    }
    results_df.to_excel(writer, 'Feedback', index=False)
    writer.save()

### TODO: Go back and fix above labels.

In [19]:
!tar -czvf "CloudFactory to Review.tar.gz" "CloudFactory to Review/"
!tar -czvf Completed.tar.gz Completed/

CloudFactory to Review/
CloudFactory to Review/marvin mwangi/
CloudFactory to Review/emily nyawira waithera/
CloudFactory to Review/damaris kwamboka okenyuri/
CloudFactory to Review/caroline kioko/
CloudFactory to Review/rasoa simiyu/
CloudFactory to Review/njoki muriithi/
CloudFactory to Review/erick karanja/
CloudFactory to Review/john oduor otieno/
CloudFactory to Review/ephantus maina/
CloudFactory to Review/joshua gichuki mwangi/
CloudFactory to Review/maina lawrence irungu/
CloudFactory to Review/moses njau/
CloudFactory to Review/vincent omondi/
CloudFactory to Review/fiona atieno/
CloudFactory to Review/vincent kipngetich/
CloudFactory to Review/isaack odhiambo otieno/
CloudFactory to Review/benice wairimu ruga/
CloudFactory to Review/clement omunga/
Completed/
Completed/marvin mwangi/
Completed/marvin mwangi/MGRS-48RWP-0115-2020-01.kml
Completed/marvin mwangi/MGRS-47QLG-0165-2020-02.kml
Completed/marvin mwangi/MGRS-49QBE-0497-2020-04.kml
Completed/marvin mwangi/MGRS-49QDF-0488